In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from tqdm import tqdm
import timm

# ⚙️ 設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset_path = "corp_augmented_data"
batch_size = 32
img_size = 224
num_classes = 2
num_epochs = 5
model_type = "efficientnet_b0"

# ✅ 資料轉換
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ✅ 載入資料集
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"類別對應: {dataset.class_to_idx}")

# ✅ 建立 EfficientNet 模型
model = timm.create_model(model_type, pretrained=True, num_classes=num_classes)
model = model.to(device)

# ✅ 損失與優化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# ✅ 訓練模型
for epoch in range(num_epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    progress_bar = tqdm(train_loader, desc=f"[EfficientNet] Epoch {epoch+1}/{num_epochs}")

    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        progress_bar.set_postfix(loss=loss.item(), acc=100 * correct / total)

    print(f"Epoch {epoch+1}: Loss={running_loss:.4f}, Acc={100 * correct / total:.2f}%")

# ✅ 儲存模型
torch.save(model.state_dict(), "efficientnet_b0_coffee_classifier.pth")
print("✅ 已儲存 EfficientNet 模型為 efficientnet_b0_coffee_classifier.pth")

# ✅ 測試
model.eval()
correct = 0
total = 0
progress_bar = tqdm(test_loader, desc="[EfficientNet] Testing")
with torch.no_grad():
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        progress_bar.set_postfix(acc=100 * correct / total)

print(f"📊 測試準確率: {100 * correct / total:.2f}%")


c:\Users\dachen\anaconda3\envs\pytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


類別對應: {'bad': 0, 'good': 1}


[EfficientNet] Epoch 1/5: 100%|██████████| 112/112 [00:15<00:00,  7.21it/s, acc=61.3, loss=1.44] 


Epoch 1: Loss=177.3252, Acc=61.34%


[EfficientNet] Epoch 2/5: 100%|██████████| 112/112 [00:15<00:00,  7.43it/s, acc=82.6, loss=0.123]


...

Epoch 4: Loss=6.9686, Acc=97.39%


[EfficientNet] Epoch 5/5: 100%|██████████| 112/112 [00:15<00:00,  7.42it/s, acc=98.9, loss=0.000215]


Epoch 5: Loss=4.1630, Acc=98.91%
✅ 已儲存 EfficientNet 模型為 efficientnet_b0_coffee_classifier.pth


[EfficientNet] Testing: 100%|██████████| 28/28 [00:01<00:00, 16.45it/s, acc=69.7]

📊 測試準確率: 69.70%
